## Magentic-One
https://microsoft.github.io/autogen/dev/user-guide/agentchat-user-guide/magentic-one.html

In [ ]:
%pip install "autogen-ext[file-surfer]==0.4.2"

## Load Azure Configuration

In [11]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client

In [12]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)


# Helper Method to see the responses per agent

In [13]:
def get_agent_responses(response):
    # ANSI escape code for bold text
    bold_start = "\033[1m"
    bold_end = "\033[0m"

    # ANSI escape code for red text
    red_start = "\033[31m"
    red_end = "\033[0m"

    print(f"{bold_start}****************Agent Responses****************{bold_end}")

    for messages in response.messages:
        source = messages.source
        print(f"{bold_start}{red_start}{source}{bold_end}{red_end}")
        print(messages.content)

## Magentic-One with Assistant Agent

In [14]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer


travel_agent = AssistantAgent(
    "TravelAgent",
    description="This agent provides a travel plan that includes travel destinations, flights, restaurants, and accommodations.",
    model_client=az_model_client,
    system_message="""
        You are a travel agent that will provide travel plans that in include travel destinations, flights, restaurants, and accommodations.
    """,
)

team = MagenticOneGroupChat([travel_agent], model_client=az_model_client)

task = """
    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.
"""
response = await Console(team.run_stream(task=task))



---------- user ----------

    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.

---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.



To answer this request we have assembled the following team:

TravelAgent: This agent provides a travel plan that includes travel destinations, flights, restaurants, and accommodations.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- The traveler wants to visit New York City for 5 days.
- The traveler has a budget of $2,500.
- The traveler will come from San

In [15]:
get_agent_responses(response)

****************Agent Responses****************
user

    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.

MagenticOneOrchestrator

We are working to address the following user request:


    Please create a travel plan.
    I want to visit New York City for 5 days. 
    I have a budget of $2,500 and will come from San Francisco.
    I want to see the Statue of Liberty, Empire State Building, and Central Park among others.



To answer this request we have assembled the following team:

TravelAgent: This agent provides a travel plan that includes travel destinations, flights, restaurants, and accommodations.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
- The traveler wants to visit New York City for 5 days.
- The traveler has a budget of $2,500.
- The traveler will come from

## Magentic-One with a MultimodalWebSurfer

In [18]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=az_model_client,
    )

team = MagenticOneGroupChat([surfer], model_client=az_model_client)
response = await Console(team.run_stream(
  task="Was there a Lakers game on January 5, 2025? Who won that game?"))

---------- user ----------
Was there a Lakers game on January 5, 2025? Who won that game?
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

Was there a Lakers game on January 5, 2025? Who won that game?


To answer this request we have assembled the following team:

WebSurfer: 
    A helpful assistant with access to a web browser.
    Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, etc., filling in form fields, etc.).
    It can also summarize the entire page, or answer questions based on the content of the page.
    It can also be asked to sleep and wait for pages to load, in cases where the pages seem to be taking a while to load.


Here is an initial fact sheet to consider:

### 1. GIVEN OR VERIFIED FACTS
- The date in question is January 5, 2025.
- The subject is a Lakers game.

### 2. FACTS TO LOOK UP
- Whether the Los Angeles Lakers had a game on January 5, 2

In [19]:
get_agent_responses(response)

****************Agent Responses****************
user
Was there a Lakers game on January 5, 2025? Who won that game?
MagenticOneOrchestrator

We are working to address the following user request:

Was there a Lakers game on January 5, 2025? Who won that game?


To answer this request we have assembled the following team:

WebSurfer: 
    A helpful assistant with access to a web browser.
    Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, etc., filling in form fields, etc.).
    It can also summarize the entire page, or answer questions based on the content of the page.
    It can also be asked to sleep and wait for pages to load, in cases where the pages seem to be taking a while to load.


Here is an initial fact sheet to consider:

### 1. GIVEN OR VERIFIED FACTS
- The date in question is January 5, 2025.
- The subject is a Lakers game.

### 2. FACTS TO LOOK UP
- Whether the Los Angeles Lakers had a game on January 

## Magentic-One with a Code Executor Agent

In [20]:
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

# Create a code executor agent that uses a Docker container to execute code.
code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
await code_executor.start()
code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)

team = MagenticOneGroupChat([code_executor_agent], model_client=az_model_client, max_turns=50)
task = f"""
You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar

"""
response = await Console(team.run_stream(task=task))

# Stop the code executor.
await code_executor.stop()

---------- user ----------

You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar


---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other actio

/home/vscode/.local/lib/python3.10/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:380: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)
/home/vscode/.local/lib/python3.10/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:390: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  response = await self._model_client.create(context, cancellation_token=cancellation_token)


---------- MagenticOneOrchestrator ----------

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python code blocks), or sh shell scripts (provided to it quoted in ```sh code blocks).


Here is an initial fact sheet to consider:

Updated fact sheet:

1. GIVEN OR VERIFIED FACTS
- The dataset is named bigfootsightings.csv.
- Each row in the dataset represents one sighting.
- The request asks for the top 10 years with the most sightings

In [21]:
get_agent_responses(response)


****************Agent Responses****************
user

You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar


MagenticOneOrchestrator

We are working to address the following user request:


You have been provided with a dataset named bigfootsightings.csv

If 1 row in the dataset is 1 sighting, What are the top 10 years with the most sightings? 
Use the date column and get the year there.
Create a column chart with the sightings in the Y-axis and years in the X-axis and save it to a PNG file and sort by descending order.
Put the value on top of each bar




To answer this request we have assembled the following team:

code_executor: A computer terminal that performs no other a

## Magentic-One with File Surfer

In [22]:
from autogen_ext.agents.file_surfer._file_surfer import FileSurfer
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console

file_path = "../Data/products/"
# Create a file surfer agent.
file_surfer = FileSurfer(
    "FileSurfer",
    model_client=az_model_client
)
file_surfer._browser.open_path(file_path)


team = MagenticOneGroupChat(
    [file_surfer],
    model_client=az_model_client, 
    max_turns=50,)

task = """What is the price of the TrailMaster X4 Tent and what are its features?"""
response = await Console(team.run_stream(task=task))


---------- user ----------
What is the price of the TrailMaster X4 Tent and what are its features?
---------- MagenticOneOrchestrator ----------

We are working to address the following user request:

What is the price of the TrailMaster X4 Tent and what are its features?


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
   - The request is asking for the price and features of the TrailMaster X4 Tent.
   
2. FACTS TO LOOK UP
   - The current price of the TrailMaster X4 Tent.
   - The features of the TrailMaster X4 Tent.
   - These can typically be found on:
     - Official website of the tent’s manufacturer.
     - Online retailers that sell camping equipment.
     - Product reviews or specifications from camping gear review websites.

3. FACTS TO DERIVE
   - There are no facts that require derivation based on the request provided.

4. EDUCATED GUESSE

/home/vscode/.local/lib/python3.10/site-packages/autogen_ext/agents/file_surfer/_file_surfer.py:128: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  create_result = await self._model_client.create(


---------- FileSurfer ----------
Path: /workspaces/azure-ai-for-developers-building-ai-agents-4283243/Data/products
Viewport position: Showing page 1 of 1.

# Index of /workspaces/azure-ai-for-developers-building-ai-agents-4283243/Data/products

| Name | Size | Date Modified |
| ---- | ---- | ------------- |
| .. (parent directory) | | |
| product_info_18.pdf | 134645 | 2025-01-13 09:27 |
| product_info_2.pdf | 134047 | 2025-01-13 09:27 |
| product_info_7.pdf | 130423 | 2025-01-13 09:27 |
| product_info_1.pdf | 134822 | 2025-01-13 09:27 |
| product_info_6.pdf | 136851 | 2025-01-13 09:27 |
| product_info_8.pdf | 134844 | 2025-01-13 09:27 |
| product_info_4.pdf | 77127 | 2025-01-13 09:27 |
| product_info_3.pdf | 135879 | 2025-01-13 09:27 |
| product_info_5.pdf | 136933 | 2025-01-13 09:27 |
| product_info_16.pdf | 139286 | 2025-01-13 09:27 |
| product_info_14.pdf | 132788 | 2025-01-13 09:27 |
| product_info_11.pdf | 133349 | 2025-01-13 09:27 |
| product_info_9.pdf | 136807 | 2025-01-13 09

In [23]:
get_agent_responses(response)

****************Agent Responses****************
user
What is the price of the TrailMaster X4 Tent and what are its features?
MagenticOneOrchestrator

We are working to address the following user request:

What is the price of the TrailMaster X4 Tent and what are its features?


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS
   - The request is asking for the price and features of the TrailMaster X4 Tent.
   
2. FACTS TO LOOK UP
   - The current price of the TrailMaster X4 Tent.
   - The features of the TrailMaster X4 Tent.
   - These can typically be found on:
     - Official website of the tent’s manufacturer.
     - Online retailers that sell camping equipment.
     - Product reviews or specifications from camping gear review websites.

3. FACTS TO DERIVE
   - There are no facts that require derivation based on the request provided.

4. EDUCATED GU